In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

In [ ]:
schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

In [ ]:
names = spark.read.schema(schema).option("sep", " ").csv("file:///C:/SparkCourse/Dataset/Marvel-names.txt")

In [ ]:
lines = spark.read.text("file:///C:/SparkCourse/Dataset/Marvel-graph.txt")

In [ ]:
# Small tweak vs. what's shown in the video: we trim each line of whitespace as that could
# throw off the counts.
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

In [ ]:
mostPopular = connections.sort(func.col("connections").asc()).first()
print(mostPopular)

In [ ]:
mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()
print(mostPopularName)

In [ ]:
print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")

In [ ]:
# Stop the session
spark.stop()